# jaxify

> The core functionality of `javiche`. Defines the `@jaxit()` decorator

In [ ]:
#| default_exp jaxify

In [ ]:
#| exporti
import numpy as np
import jax
import autograd
import jax.numpy as jnp
from typing import List
from functools import lru_cache, wraps

In [ ]:
#| export
# from https://gist.github.com/Susensio/61f4fee01150caaac1e10fc5f005eb75
def np_cache(*args, **kwargs): 
    """LRU cache implementation for functions whose FIRST parameter is a numpy array
    >>> array = np.array([[1, 2, 3], [4, 5, 6]])
    >>> @np_cache(maxsize=256)
    ... def multiply(array, factor):
    ...     print("Calculating...")
    ...     return factor*array
    >>> multiply(array, 2)
    Calculating...
    array([[ 2,  4,  6],
           [ 8, 10, 12]])
    >>> multiply(array, 2)
    array([[ 2,  4,  6],
           [ 8, 10, 12]])
    >>> multiply.cache_info()
    CacheInfo(hits=1, misses=1, maxsize=256, currsize=1)
    
    """
    def decorator(function):
        @wraps(function)
        def wrapper(*args, **kwargs):
            mod_args = []
            for i,arg in enumerate(args): # modified to allow arbitrary amounts of arrays
              if isinstance(arg, np.ndarray) or isinstance(arg, jax.Array):
                mod_args.append(array_to_tuple(arg))
              else:
                mod_args.append(arg)
            return cached_wrapper(*mod_args, **kwargs)

        @lru_cache(*args, **kwargs)
        def cached_wrapper(hashable_array, *args, **kwargs):
            array = np.array(hashable_array)
            return function(array, *args, **kwargs)

        def array_to_tuple(np_array):
            """Iterates recursivelly."""
            #print(type(np_array))
            if isinstance(np_array, jax.Array):
              np_array = np.asarray(np_array)
            try:
              return tuple(array_to_tuple(_) for _ in np_array)
            except TypeError:
              return np_array

        # copy lru_cache attributes over too
        wrapper.cache_info = cached_wrapper.cache_info
        wrapper.cache_clear = cached_wrapper.cache_clear

        return wrapper

    return decorator

In [ ]:
#| export
def as_numpy(x):
  def as_numpy_map(a):
    if isinstance(a, jnp.ndarray):
      return np.asarray(a)
    else:
      return a
  return jax.tree_util.tree_map(as_numpy_map, x)


def as_jax(x):
  def as_jax_map(a):
    if isinstance(a, np.ndarray):
      return jnp.asarray(a)
    else:
      return a
  return jax.tree_util.tree_map(as_jax_map, x)

In [ ]:
#| export
def jaxit(
  cache: bool = False
  ):
  """
    make a function that internally uses autograd compatible to jax gradient calculations

    Attention: only a single output variable is supported
  """
  def identity_decorator(fn):
    return fn
  
  caching_decorator = identity_decorator
  if cache:
    caching_decorator = np_cache()

  def inner(function):

    @jax.custom_vjp
    @caching_decorator
    def jaxed(*args):
      return as_jax(function(*as_numpy(args)))

    @caching_decorator
    def f_fwd(*args):
      args = as_numpy(args)
      argnums = tuple(i for i, _ in enumerate(args))

      def f_ag_tupled(*args):
        ans = function(*args)
        if isinstance(ans, tuple):
          return autograd.builtins.tuple(ans)
        else:
          return ans

      vjp_f, ans = autograd.make_vjp(f_ag_tupled, argnums)(*args)
      return as_jax(ans), jax.tree_util.Partial(vjp_f)


    def f_rev(vjp_f, g):
      g = as_numpy(g)
      return as_jax(vjp_f(g))


    jaxed.defvjp(f_fwd, f_rev)

    return jaxed
  return inner


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()